<a href="https://colab.research.google.com/github/erassmirza/ML_termProject/blob/main/Project_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install pycoingecko
from pycoingecko import CoinGeckoAPI
import pandas as pd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Coingecko

In [2]:
cg = CoinGeckoAPI()

## Extract Data

In [3]:
data = cg.get_coin_market_chart_by_id(id='ethereum',
                                   vs_currency='usd',
                                   days = 10000)
df = pd.DataFrame(data)
prices = list(df['prices'])
market_caps = list(df['market_caps'])
total_volumes = list(df['total_volumes'])

timestamp = []
price = []
market_cap = []
total_volume = []

for i in range(len(prices)):
  timestamp.append(prices[i][0])
  price.append(prices[i][1])
  market_cap.append(market_caps[i][1])
  total_volume.append(total_volumes[i][1])

data = pd.DataFrame([timestamp, price, market_cap, total_volume]).T
data.rename(columns = {0:'timestamp',
                             1:'price',
                             2:'market_cap',
                             3:'total_volume'}, inplace = True)
data['timestamp'] = pd.to_datetime(data['timestamp'], unit="ms")
data

,timestamp,price,market_cap,total_volume
0,2015-08-07 00:00:00,2.831620,0.000000e+00,9.062200e+04
1,2015-08-08 00:00:00,1.330750,8.033948e+07,3.680700e+05
2,2015-08-10 00:00:00,0.687586,4.155631e+07,4.004641e+05
3,2015-08-11 00:00:00,1.067379,6.453901e+07,1.518998e+06
4,2015-08-12 00:00:00,1.256613,7.601326e+07,2.073893e+06
...,...,...,...,...
2875,2023-06-22 00:00:00,1892.899572,2.275816e+11,1.229479e+10
2876,2023-06-23 00:00:00,1874.177947,2.253811e+11,9.842203e+09
2877,2023-06-24 00:00:00,1891.345008,2.274194e+11,9.303341e+09
2878,2023-06-25 00:00:00,1875.059962,2.254489e+11,4.889179e+09


## Percentage Change

In [4]:
data['price_change'] = data['price'].pct_change()
data['market_cap_change'] = data['market_cap'].pct_change()
data['total_volume_change'] = data['total_volume'].pct_change()

## Moving Average with period of 2 (current and previous day)

In [5]:
# data['price'].rolling(window=2).mean()
data['price_ma'] = data['price'].rolling(window=2).mean()
data['market_cap_ma'] = data['market_cap'].rolling(window=2).mean()
data['total_volume_ma'] = data['total_volume'].rolling(window=2).mean()

## Price to market capital ratio

In [6]:
data['price_to_marketcap'] = data['price'] / data['market_cap']

## Price to volume ratio

In [7]:
data['price_to_volume'] = data['price'] / data['total_volume']

## Market capital to volume ratio

In [8]:
data['marketcap_to_volume'] = data['market_cap'] / data['total_volume']

## Weekly/Monthly Return

In [9]:
data['weekly_retun'] = data['price'].pct_change(periods=7)
data['monthly_retun'] = data['price'].pct_change(periods=30)

## Price percentage change from all-time high

In [10]:
data['price_pct_change_from_high'] = (data['price'] - data['price'].max()) / data['price'].max()

## Price percentage change from all-time low

In [11]:
data['price_pct_change_from_low'] = (data['price'] - data['price'].min()) / data['price'].min()

## Price volatility

In [12]:
data['price_volatility'] = data['price'].rolling(window=2).std()

In [13]:
data

,timestamp,price,market_cap,total_volume,price_change,market_cap_change,total_volume_change,price_ma,market_cap_ma,total_volume_ma,price_to_marketcap,price_to_volume,marketcap_to_volume,weekly_retun,monthly_retun,price_pct_change_from_high,price_pct_change_from_low,price_volatility
0,2015-08-07 00:00:00,2.831620,0.000000e+00,9.062200e+04,NaN,NaN,NaN,NaN,NaN,NaN,inf,3.124650e-05,0.000000,NaN,NaN,-0.999412,5.539861,NaN
1,2015-08-08 00:00:00,1.330750,8.033948e+07,3.680700e+05,-0.530039,inf,3.061597,2.081185,4.016974e+07,2.293460e+05,1.656409e-08,3.615481e-06,218.272272,NaN,NaN,-0.999724,2.073478,1.061275
2,2015-08-10 00:00:00,0.687586,4.155631e+07,4.004641e+05,-0.483310,-0.482741,0.088011,1.009168,6.094789e+07,3.842670e+05,1.654588e-08,1.716972e-06,103.770380,NaN,NaN,-0.999857,0.588036,0.454786
3,2015-08-11 00:00:00,1.067379,6.453901e+07,1.518998e+06,0.552358,0.553050,2.793095,0.877482,5.304766e+07,9.597311e+05,1.653851e-08,7.026861e-07,42.487875,NaN,NaN,-0.999778,1.465200,0.268554
4,2015-08-12 00:00:00,1.256613,7.601326e+07,2.073893e+06,0.177289,0.177788,0.365303,1.161996,7.027613e+07,1.796446e+06,1.653150e-08,6.059199e-07,36.652445,NaN,NaN,-0.999739,1.902252,0.133809
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2875,2023-06-22 00:00:00,1892.899572,2.275816e+11,1.229479e+10,0.057373,0.058219,0.556252,1841.545006,2.213213e+11,1.009753e+10,8.317453e-09,1.539594e-07,18.510407,0.146742,0.040969,-0.606875,4370.808621,72.626323
2876,2023-06-23 00:00:00,1874.177947,2.253811e+11,9.842203e+09,-0.009890,-0.009669,-0.199482,1883.538759,2.264814e+11,1.106850e+10,8.315596e-09,1.904226e-07,22.899456,0.125648,0.010778,-0.610763,4327.569475,13.238188
2877,2023-06-24 00:00:00,1891.345008,2.274194e+11,9.303341e+09,0.009160,0.009044,-0.054750,1882.761478,2.264003e+11,9.572772e+09,8.316550e-09,2.032974e-07,24.444918,0.101940,0.050318,-0.607198,4367.218227,12.138945
2878,2023-06-25 00:00:00,1875.059962,2.254489e+11,4.889179e+09,-0.008610,-0.008665,-0.474471,1883.202485,2.264342e+11,7.096260e+09,8.317005e-09,3.835122e-07,46.111820,0.086127,0.038351,-0.610580,4329.606561,11.515267
